In [2]:
import pandas as pd
from model.core import SimulationConfig, run_simulation

# --- Build a minimal single-round debug config ---
cfg = SimulationConfig(
    debug=True,                # ENABLE DEBUG LOGGING
    adventurer="DG",
    star=10,
    weapon="Nashir",

    # Disable other skills for clean debug
    use_extra_end_bolts=False,
    basic_atk_bolt_level=0,
    five_bolts_from_round6=False,

    rounds=1,
    basic_hits_per_round=1,   # only basic hit -> flame proc -> clean view
    seed=123,
    
    # Example multipliers (adjust to match your test)
    base_global_bonus=0.16,
    base_global_skill_bonus=0.16,
    base_global_lightning_bonus=0.0,

    base_inbattle_bonus=0.0,
    base_inbattle_basic_bonus=0.0,
    base_inbattle_skill_bonus=0.0,
    base_inbattle_lightning_bonus=0.0,

    base_final_bonus=0.26,
    base_final_skill_bonus=0.15,
    base_final_lightning_bonus=0.25,
)

# --- Run sim with debug mode ---
state, log = run_simulation(cfg, with_log=True)

# --- Convert debug logs to DataFrame ---
df_debug = pd.DataFrame(state.debug_logs)
df_debug


,round,hit_index,type,tags,coeff,atk_eff,K,global_total,inbattle_total,final_total,damage,lc_stacks,lc_bonus
0,1,1,basic,[basic],1.00,1150.0,12.0,0.21,0.0,0.26,21039.48,0,0.0
1,1,1,dragon_flame,"[dragon_flame, skill]",1.80,1150.0,50.0,0.37,0.0,0.41,199930.95,0,0.0
2,1,1,dragon_breath,"[dragon_flame, skill]",12.00,1150.0,50.0,0.42,0.0,0.41,1381518.00,0,0.0
3,1,1,other_skill,"[bolt, lightning, skill]",1.11,1150.0,50.0,0.62,0.0,0.66,171638.19,0,0.0


In [3]:
import pandas as pd
from model.core import SimulationConfig, run_simulation

# Tiny debug scenario: DG + Nashir, only extra-end bolts active
cfg = SimulationConfig(
    debug=True,
    adventurer="DG",
    star=0,                   # keep DG at 0* to avoid Breath complication
    weapon="Nashir",

    combo_mastery=False,

    # Turn ON only extra end-of-round bolts; others OFF
    use_extra_end_bolts=True,
    extra_end_bolts_count=3,   # base 3 bolts per round
    basic_atk_bolt_level=0,
    five_bolts_from_round6=False,

    # Lightning Charge
    lightning_charge_step=0.06,  # 6% per bolt
    # Multiple Lightning
    multiple_lightning_factor=2, # doubles normal lightning bolts

    rounds=3,                    # 3 rounds to see 1 reset
    basic_hits_per_round=1,      # minimal basics to keep log small
    seed=42,

    # Keep all other multipliers 0 so numbers are easy to reason about
    base_global_bonus=0.0,
    base_inbattle_bonus=0.0,
    base_final_bonus=0.0,
    base_global_skill_bonus=0.0,
    base_global_lightning_bonus=0.0,
    base_final_skill_bonus=0.0,
    base_final_lightning_bonus=0.0,
    base_inbattle_basic_bonus=0.0,
    base_inbattle_skill_bonus=0.0,
    base_inbattle_lightning_bonus=0.0,
)

state, round_log = run_simulation(cfg, with_log=True)

df = pd.DataFrame(state.debug_logs)
df.head()


,round,hit_index,type,tags,coeff,atk_eff,K,global_total,inbattle_total,final_total,damage,lc_stacks,lc_bonus
0,1,1,basic,[basic],1.00,1150.0,12.0,0.0,0.00,0.0,13800.0,0,0.00
1,1,1,dragon_flame,"[dragon_flame, skill]",0.90,1150.0,50.0,0.0,0.00,0.0,51750.0,0,0.00
2,1,1,other_skill,"[bolt, lightning, skill]",0.36,1150.0,50.0,0.0,0.00,0.0,20700.0,0,0.00
3,1,1,other_skill,"[bolt, lightning, skill]",1.11,1150.0,50.0,0.0,0.06,0.0,67654.5,1,0.06
4,1,1,other_skill,"[bolt, lightning, skill]",1.11,1150.0,50.0,0.0,0.12,0.0,71484.0,2,0.12


In [4]:
# Select only bolt hits
df_bolts = df[df["tags"].apply(lambda ts: "bolt" in ts)]
df_bolts[["round", "hit_index", "tags", "damage", "lc_stacks", "lc_bonus"]]


,round,hit_index,tags,damage,lc_stacks,lc_bonus
2,1,1,"[bolt, lightning, skill]",20700.0,0,0.00
3,1,1,"[bolt, lightning, skill]",67654.5,1,0.06
4,1,1,"[bolt, lightning, skill]",71484.0,2,0.12
5,1,1,"[bolt, lightning, skill]",75313.5,3,0.18
6,1,1,"[bolt, lightning, skill]",25668.0,4,0.24
7,1,1,"[bolt, lightning, skill]",26910.0,5,0.30
8,1,1,"[bolt, lightning, skill]",28152.0,6,0.36
11,2,1,"[bolt, lightning, skill]",115126.5,7,0.42
12,2,1,"[bolt, lightning, skill]",119991.0,8,0.48
13,2,1,"[bolt, lightning, skill]",124855.5,9,0.54


In [5]:
# Number of bolt hits per round
df_bolts.groupby("round").size()


round
1     7
2     7
3    13
dtype: int64

In [6]:
print("Final LC stacks:", state.lightning_charge_stacks)
print("Final LC bonus:", state.dynamic_inbattle_lightning_bonus)


Final LC stacks: 6
Final LC bonus: 0.36


In [1]:
from model.core import SimulationConfig, run_simulation

# --- Leo 10★ + Nashir + upgraded basic atk bolt ---
cfg_leo10 = SimulationConfig(
    debug=True,
    adventurer="Leo",
    star=10,
    weapon="Nashir",
    combo_mastery=True,          # so combo stacks affect bolts

    # Lightning skills
    use_extra_end_bolts=False,   # keep off to make the log simpler
    extra_end_bolts_count=0,
    basic_atk_bolt_level=2,      # 0 = off, 1 = base, 2 = upgraded
    five_bolts_from_round6=False,

    # Battle length / RNG
    rounds=5,
    basic_hits_per_round=3,      # 3 basics per round keeps output readable
    seed=42,                     # fixed seed for reproducibility

    # No external buffs: we want to see Leo’s own effects clearly
    base_global_bonus=0.0,
    base_inbattle_bonus=0.0,
    base_final_bonus=0.0,
    base_global_skill_bonus=0.0,
    base_global_lightning_bonus=0.0,
    base_global_ninjutsu_bonus=0.0,
    base_final_skill_bonus=0.0,
    base_final_lightning_bonus=0.0,
    base_inbattle_basic_bonus=0.0,
    base_inbattle_skill_bonus=0.0,
    base_inbattle_lightning_bonus=0.0,

    # Lightning Charge
    lightning_charge_step=0.06,  # or 0.10 if you want the upgraded LC
    multiple_lightning_factor=1,

    # No Ezra, no artifact in this test
    use_ezra_ring=False,
    ezra_final_light_bonus=0.20,
    artifact=None,
    artifact_level=0,
)

state10, round_log10 = run_simulation(cfg_leo10, with_log=True)

print("=== Leo 10★ test ===")
print("dynamic_global_ninjutsu_bonus:", state10.dynamic_global_ninjutsu_bonus)
print("per-round damage:")
for row in round_log10:
    print(row)


=== Leo 10★ test ===
dynamic_global_ninjutsu_bonus: 0.6
per-round damage:
{'round': 1, 'basic': 81840.0, 'ninjutsu': 495000.0, 'hurricane': 1100000.0, 'ultimate': 0.0, 'bolt': 344335.19999999995, 'other': 0.0}
{'round': 2, 'basic': 97680.0, 'ninjutsu': 471350.0, 'hurricane': 0.0, 'ultimate': 0.0, 'bolt': 599441.04, 'other': 0.0}
{'round': 3, 'basic': 113520.0, 'ninjutsu': 609400.0, 'hurricane': 1100000.0, 'ultimate': 825000.0, 'bolt': 7633543.103999998, 'other': 0.0}
{'round': 4, 'basic': 129360.0, 'ninjutsu': 1251360.0, 'hurricane': 0.0, 'ultimate': 0.0, 'bolt': 2135208.767999999, 'other': 0.0}
{'round': 5, 'basic': 145200.0, 'ninjutsu': 1243440.0, 'hurricane': 1760000.0, 'ultimate': 0.0, 'bolt': 6529604.928000001, 'other': 0.0}


In [2]:
cfg_leo9 = cfg_leo10
cfg_leo9 = SimulationConfig(
    **{**cfg_leo10.__dict__, "star": 9}   # copy cfg_leo10 but star=9
)

state9, round_log9 = run_simulation(cfg_leo9, with_log=True)

print("\n=== Leo 9★ (no global ninjutsu buff) ===")
print("dynamic_global_ninjutsu_bonus:", state9.dynamic_global_ninjutsu_bonus)
for row in round_log9:
    print(row)



=== Leo 9★ (no global ninjutsu buff) ===
dynamic_global_ninjutsu_bonus: 0.0
{'round': 1, 'basic': 81840.0, 'ninjutsu': 495000.0, 'hurricane': 1100000.0, 'ultimate': 0.0, 'bolt': 344335.19999999995, 'other': 0.0}
{'round': 2, 'basic': 97680.0, 'ninjutsu': 471350.0, 'hurricane': 0.0, 'ultimate': 0.0, 'bolt': 599441.04, 'other': 0.0}
{'round': 3, 'basic': 113520.0, 'ninjutsu': 609400.0, 'hurricane': 1100000.0, 'ultimate': 495000.0, 'bolt': 5796267.840000001, 'other': 0.0}
{'round': 4, 'basic': 129360.0, 'ninjutsu': 782100.0, 'hurricane': 0.0, 'ultimate': 0.0, 'bolt': 1334505.4800000004, 'other': 0.0}
{'round': 5, 'basic': 145200.0, 'ninjutsu': 777150.0, 'hurricane': 1100000.0, 'ultimate': 0.0, 'bolt': 4081003.079999999, 'other': 0.0}


In [5]:
from model.core import SimulationConfig, run_simulation, print_damage_breakdown

cfg_dg = SimulationConfig(
    adventurer="DG",
    star=10,
    weapon="Nashir",
    rounds=10,
    basic_hits_per_round=5,
    seed=42,
)

cfg_leo = SimulationConfig(
    adventurer="Leo",
    star=10,
    weapon="Nashir",
    rounds=10,
    basic_hits_per_round=5,
    seed=42,
)

state_dg = run_simulation(cfg_dg, with_log=False)
state_leo = run_simulation(cfg_leo, with_log=False)

print_damage_breakdown(state_dg, adventurer="DG")
print()
print_damage_breakdown(state_leo, adventurer="Leo")


=== Damage breakdown for DG ===
Total damage: 32258811.00
- bolt      :  12722151.00  ( 39.4%)
- flame     :  11819700.00  ( 36.6%)
- breath    :   6348000.00  ( 19.7%)
- basic     :   1368960.00  (  4.2%)

=== Damage breakdown for Leo ===
Total damage: 36275778.00
- bolt      :  14383578.00  ( 39.7%)
- ninjutsu  :   9627200.00  ( 26.5%)
- hurricane :   7480000.00  ( 20.6%)
- ultimate  :   3465000.00  (  9.6%)
- basic     :   1320000.00  (  3.6%)


In [4]:
from model.core import SimulationConfig, run_simulation, print_damage_breakdown

cfg_leo = SimulationConfig(
    adventurer="Leo",
    star=10,
    weapon="Nashir",
    rounds=10,
    basic_hits_per_round=5,
    seed=42,
    basic_atk_bolt_level=2,
    lightning_charge_step=0.06,
)

state_leo, log_leo = run_simulation(cfg_leo, with_log=True)
print_damage_breakdown(state_leo, adventurer="Leo")
for row in log_leo:
    print(row)


=== Damage breakdown for Leo ===
Total damage: 90548099.84
- bolt      :  69222399.84  ( 76.4%)
- ninjutsu  :   9060700.00  ( 10.0%)
- hurricane :   7480000.00  (  8.3%)
- ultimate  :   3465000.00  (  3.8%)
- basic     :   1320000.00  (  1.5%)
{'round': 1, 'basic': 132000.0, 'ninjutsu': 720500.0, 'hurricane': 1100000.0, 'ultimate': 0.0, 'bolt': 867583.1999999998, 'other': 0.0}
{'round': 2, 'basic': 132000.0, 'ninjutsu': 660000.0, 'hurricane': 0.0, 'ultimate': 0.0, 'bolt': 3701464.8000000003, 'other': 0.0}
{'round': 3, 'basic': 132000.0, 'ninjutsu': 737000.0, 'hurricane': 1100000.0, 'ultimate': 825000.0, 'bolt': 10336048.8, 'other': 0.0}
{'round': 4, 'basic': 132000.0, 'ninjutsu': 818400.0, 'hurricane': 0.0, 'ultimate': 0.0, 'bolt': 5396159.999999996, 'other': 0.0}
{'round': 5, 'basic': 132000.0, 'ninjutsu': 1020800.0, 'hurricane': 1760000.0, 'ultimate': 0.0, 'bolt': 8716794.239999998, 'other': 0.0}
{'round': 6, 'basic': 132000.0, 'ninjutsu': 1205600.0, 'hurricane': 0.0, 'ultimate': 132

In [6]:
state_dg, log_dg = run_simulation(cfg_dg, with_log=True)
state_leo, log_leo = run_simulation(cfg_leo, with_log=True)

print_damage_breakdown(state_dg, adventurer="DG")
print()
print_damage_breakdown(state_leo, adventurer="Leo")

print("\nDG round log:")
for row in log_dg:
    print(row)

print("\nLeo round log:")
for row in log_leo:
    print(row)


=== Damage breakdown for DG ===
Total damage: 32258811.00
- bolt      :  12722151.00  ( 39.4%)
- flame     :  11819700.00  ( 36.6%)
- breath    :   6348000.00  ( 19.7%)
- basic     :   1368960.00  (  4.2%)

=== Damage breakdown for Leo ===
Total damage: 36275778.00
- bolt      :  14383578.00  ( 39.7%)
- ninjutsu  :   9627200.00  ( 26.5%)
- hurricane :   7480000.00  ( 20.6%)
- ultimate  :   3465000.00  (  9.6%)
- basic     :   1320000.00  (  3.6%)

DG round log:
{'round': 1, 'basic': 79350.0, 'flame': 595125.0, 'breath': 897000.0, 'bolt': 325610.99999999994, 'other': 0.0}
{'round': 2, 'basic': 120750.0, 'flame': 1216125.0, 'breath': 0.0, 'bolt': 297804.00000000006, 'other': 0.0}
{'round': 3, 'basic': 120750.0, 'flame': 905625.0, 'breath': 1311000.0, 'bolt': 3151161.0, 'other': 0.0}
{'round': 4, 'basic': 158010.0, 'flame': 1495575.0, 'breath': 0.0, 'bolt': 615342.0, 'other': 0.0}
{'round': 5, 'basic': 138000.0, 'flame': 1035000.0, 'breath': 1380000.0, 'bolt': 740922.0, 'other': 0.0}
{'ro

In [3]:
from model.core import SimulationConfig, run_simulation

def count_bolts(state):
    # Count all hits that have the "bolt" tag
    return sum(1 for h in state.debug_logs if "bolt" in h["tags"])


def bolt_chance_test(bolt_level=1, rounds=200, basic_hits_per_round=5, seed=123):
    """
    Compare how many basic-attack bolts DG vs Leo produce
    when ONLY the Basic Attack Bolt skill is active.
    """
    base_kwargs = dict(
        weapon=None,                 # no Nashir → only basic-atk bolt exists
        combo_mastery=False,
        use_extra_end_bolts=False,
        extra_end_bolts_count=0,
        five_bolts_from_round6=False,
        lightning_charge_step=0.0,   # no Lightning Charge
        artifact=None,
        artifact_level=0,
        debug=True,                  # enable debug_logs so we can count hits
        rounds=rounds,
        basic_hits_per_round=basic_hits_per_round,
        seed=seed,
        basic_atk_bolt_level=bolt_level,
    )

    # Dragon Girl config
    cfg_dg = SimulationConfig(
        adventurer="DG",
        star=0,
        **base_kwargs,
    )

    # Leo config (star doesn't matter for bolt chance, only adventurer name)
    cfg_leo = SimulationConfig(
        adventurer="Leo",
        star=0,
        **base_kwargs,
    )

    # Run simulations
    state_dg = run_simulation(cfg_dg, with_log=False)
    state_leo = run_simulation(cfg_leo, with_log=False)

    # Count bolts
    bolts_dg = count_bolts(state_dg)
    bolts_leo = count_bolts(state_leo)

    num_basics = rounds * basic_hits_per_round  # 1 bolt roll per basic/combo

    print(f"=== Basic Attack Bolt level {bolt_level} ===")
    print(f"Total basic attacks: {num_basics}")
    print(f"DG  bolts: {bolts_dg}  ({bolts_dg / num_basics:.3f} per basic)")
    print(f"Leo bolts: {bolts_leo}  ({bolts_leo / num_basics:.3f} per basic)")
    print()


# Run tests for level 1 and level 2
bolt_chance_test(bolt_level=1)
bolt_chance_test(bolt_level=2)


=== Basic Attack Bolt level 1 ===
Total basic attacks: 1000
DG  bolts: 1228  (1.228 per basic)
Leo bolts: 1696  (1.696 per basic)

=== Basic Attack Bolt level 2 ===
Total basic attacks: 1000
DG  bolts: 1722  (1.722 per basic)
Leo bolts: 1978  (1.978 per basic)



In [ ]:
from model.core import SimulationConfig, run_simulation

# ---------------------------------
# Helper: pretty damage breakdown
# ---------------------------------
def print_daji_breakdown(state, label="Daji"):
    # Use getattr for the new fields so it doesn't crash if a name is missing
    components = [
        ("Basic",      getattr(state, "dmg_basic", 0.0)),
        ("Bolt",       getattr(state, "dmg_bolt", 0.0)),
        ("Artifact",   getattr(state, "dmg_artifact", 0.0)),
        ("Demonic",    getattr(state, "dmg_demonic", 0.0)),
        ("FoxFlame",   getattr(state, "dmg_fox_flame", 0.0)),
        ("Ultimate",   getattr(state, "dmg_ultimate", 0.0)),
        ("Other",      getattr(state, "dmg_other", 0.0)),
    ]

    total = sum(v for _, v in components)
    print(f"====== {label} ======")
    print(f"Total damage: {total:,.0f}")
    if total <= 0:
        return

    for name, val in components:
        if val <= 0:
            continue
        share = 100.0 * val / total
        print(f"{name:10s}: {val:,.0f}  ({share:5.1f}%)")


# ---------------------------------
# Shared config for Daji
# ---------------------------------
base_cfg_kwargs = dict(
    adventurer="Daji",
    weapon="Nashir",
    combo_mastery=False,

    # Lightning skills
    use_extra_end_bolts=True,
    extra_end_bolts_count=3,
    basic_atk_bolt_level=1,     # base bolt skill on

    five_bolts_from_round6=False,

    # Battle length / RNG
    rounds=15,
    basic_hits_per_round=5,
    seed=42,

    # Base multipliers – you can tweak these to match your usual test setup
    base_global_bonus=0.0,
    base_inbattle_bonus=0.0,
    base_final_bonus=0.0,
    base_global_skill_bonus=0.0,
    base_global_lightning_bonus=0.0,
    base_global_ninjutsu_bonus=0.0,
    base_final_skill_bonus=0.0,
    base_final_lightning_bonus=0.0,
    base_inbattle_basic_bonus=0.0,
    base_inbattle_skill_bonus=0.0,
    base_inbattle_lightning_bonus=0.0,

    # Lightning Charge off for this sanity check
    lightning_charge_step=0.0,
    multiple_lightning_factor=1,

    # Ezra off so we clearly see Daji’s own effects
    use_ezra_ring=False,
    ezra_final_light_bonus=0.20,

    # Artifact – turn on if you also want to see 5★ fox-form lightning-as-demonic interaction
    artifact=None,        # or "ArcaneTome"
    artifact_level=0,     # 0 = off, 1/2 = on
)

# 0★ and 10★ configs
cfg_daji_0  = SimulationConfig(star=0,  **base_cfg_kwargs)
cfg_daji_10 = SimulationConfig(star=10, **base_cfg_kwargs)

# ---------------------------------
# Run simulations (15 rounds)
# ---------------------------------
state_0  = run_simulation(cfg_daji_0)      # with_log=False → returns BattleState
state_10 = run_simulation(cfg_daji_10)

# ---------------------------------
# Print damage breakdowns
# ---------------------------------
print_daji_breakdown(state_0,  "Daji 0★ – 15 rounds")
print()
print_daji_breakdown(state_10, "Daji 10★ – 15 rounds")
